In [1]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [17]:
intents = [
    {
        'tag':'greeting',
        'patterns':['hi', 'hello', 'hey', 'whats up', 'how are you'],
        'response':['hi there', 'hello', 'hey', 'Nothing much', 'Im fine', 'Thank you']
    },
   {
    'tag': 'send_off',
    'patterns': [
        'bye', 
        'goodbye', 
        'see you later', 
        'take care', 
        'catch you later',
        'have a good day',
        'farewell',
        'talk to you later',
        'until next time',
        'so long'
    ],
    'response': [
        'Goodbye!',
        'See you later!',
        'Take care!',
        'Have a great day!',
        'Bye! Let\'s chat again soon.',
        'Farewell!',
        'Until next time!',
        'Catch you later!',
        'So long!',
        'It was nice talking to you!'
    ]
},
   {
    'tag': 'thanks',
    'patterns': [
        'thank you', 
        'thanks', 
        'thanks a lot', 
        'appreciate it',
        'thank you so much',
        'many thanks',
        'cheers',
        'much obliged',
        'thanks a ton',
        'thanks a bunch',
        'you\'re awesome',
        'i owe you one'
    ],
    'response': [
        'You\'re welcome!',
        'Happy to help!',
        'Anytime!',
        'My pleasure!',
        'No problem at all!',
        'Glad I could assist!',
        'Don\'t mention it!',
        'You got it!',
        'Of course!',
        'That\'s what I\'m here for!'
    ]
},
   {
    'tag': 'about',
    'patterns': [
        'what are you?',
        'who created you?',
        'tell me about yourself',
        'what can you do?',
        'what is your purpose?',
        'how do you work?',
        'are you human?',
        'who are you?',
        'what\'s your function?',
        'can you explain yourself?',
        'what kind of bot are you?'
    ],
    'response': [
        'I\'m an AI chatbot designed to assist with information and conversations',
        'I\'m a virtual assistant here to help answer questions and chat with you',
        'My purpose is to assist you with various queries and tasks',
        'I\'m a language model created to communicate and help with information',
        'I\'m a digital assistant that learns from interactions to help users',
        'I\'m here to answer questions and help with whatever you need!',
        'I\'m an artificial intelligence programmed for conversation and support',
        'My main function is to provide helpful responses to your questions'
    ]
},
   {
    'tag': 'age',
    'patterns': [
        'how old are you?',
        'what is your age?',
        'when were you created?',
        'what\'s your birth year?',
        'are you old?',
        'do you have an age?',
        'how long have you existed?',
        'what year were you made?',
        'age?',
        'when is your birthday?',
        'how young are you?',
        'are you a new AI?'
    ],
    'response': [
        'I\'m a digital being, so age works differently for me!',
        'I was created recently to help with modern questions so i dont have any age',
        'Age is just a number for an AI! How can I assist you today?',
        'I don\'t have birthdays, but I\'m always learning new things',
        'My knowledge is current, but I don\'t track time like humans do',
         'Time works differently for AIs - let\'s focus on how I can help you',
        'I\'m as young as my latest update! What can I do for you?',
        'No candles on my cake, but I\'m here 24/7 to assist you'
    ]
},
   {
    'tag': 'help',
    'patterns': [
        'help me',
        'i need help',
        'can you help',
        'assist me',
        'support needed',
        'what should I do?',
        'how does this work?',
        'guide me',
        'i need assistance',
        'can you explain?',
        'show me how',
        'what do I do now?',
        'trouble with this',
        'not working',
        'stuck here',
        'confused'
    ],
    'response': [
        'How can I assist you?',
        'What do you need help with?',
        'I\'m here to help! Please describe your issue',
        'Sure, what would you like assistance with?',
        'Let me help you with that. What\'s the problem?',
        'No worries! Tell me what you need',
        'I\'ll do my best to help. What\'s the challenge?',
        'Feel free to ask me anything - how can I help?',
        'Let\'s solve this together. What do you need?',
        'How can I support you today?'
    ]
},
   {  
    'tag': 'budget',  
    'patterns': [  
        'how to create a budget',  
        'budgeting tips',  
        'help with my budget',  
        'manage expenses',  
        'track spending',  
        'save money',  
        'stick to a budget',  
        'budget planning',  
        'financial planning',  
        'reduce costs',  
        'cut expenses',  
        'monthly budget',  
        'staying within budget',  
        'i need help budgeting',  
        'cant manage my budget'  
    ],  
    'response': [  
        'Start by tracking all your expenses for a month to understand spending habits!',  
        'Try the 50/30/20 rule: 50% needs, 30% wants, 20% savings/debt repayment.',  
        'Prioritize essential expenses first, then allocate funds for savings and goals.',  
        'Use budgeting apps like Mint or YNAB to automate tracking and stay organized.',  
        'Review your budget weekly and adjust categories where you overspend.',  
        'Set clear financial goals to stay motivated—even small savings add up over time!',  
        'Try the envelope method: allocate cash to categories and stop when it’s gone.',  
        'Look for recurring expenses you can reduce, like subscriptions or dining out.',  
        'Automate savings by setting up direct transfers to a separate account.',  
        'Celebrate milestones—like sticking to your budget for 30 days—to stay encouraged!'  
    ]  
},

    {  
    'tag': 'credit_score',  
    'patterns': [  
        'how to check my credit score',  
        'improve credit score',  
        'what is a credit score',  
        'good credit score range',  
        'why is my credit score low',  
        'raise credit score',  
        'credit score factors',  
        'fix bad credit',  
        'what affects credit score',  
        'check credit report',  
        'how to build credit',  
        'credit score too low',  
        'increase credit score fast',  
        'credit monitoring',  
        'my credit is bad'  
    ],  
    'response': [  
        'Check your credit report free at AnnualCreditReport.com - review it for errors!',  
        'Key factors: payment history (35%), credit utilization (30%), credit age (15%), credit mix (10%), new credit (10%)',  
        'Most scores range 300-850. 670+ is good, 740+ is excellent.',  
        'To improve: pay bills on time, keep credit utilization below 30%, avoid new hard inquiries',  
        'Dispute errors on your report and pay down high balances to boost your score',  
        'Building credit? Consider a secured credit card or becoming an authorized user',  
        'Avoid closing old accounts - longer credit history improves your score',  
        'Limit new credit applications - each hard inquiry can temporarily lower your score',  
        'Use credit monitoring services to track changes and detect fraud early',  
        'Consistency matters! It takes 3-6 months of good habits to see significant changes'  
    ]  
},

   {
    'tag': 'weather',
    'patterns': [
        'what\'s the weather?',
        'is it raining?',
        'weather forecast',
        'temperature today',
        'will it snow tomorrow?',
        'humidity level',
        'is it sunny outside?',
        'current weather',
        'how hot is it?',
        'should I take an umbrella?',
        'wind speed',
        'weather update',
        'is it cold today?'
    ],
    'response': [
        "I don't have access to real-time weather data.",
        "I can't check current weather conditions, but you might want to try a weather app!",
        "Weather information isn't available through me - try a dedicated service.",
        "I'm not connected to weather systems. Check your local forecast online!",
        "Unfortunately I can't provide live weather updates.",
        "For weather info, you'll need to consult a weather service - I don't have those capabilities.",
        "My systems don't include meteorological data. Try searching online!",
        "I wish I could help with weather, but I don't have that functionality."
    ]
},

]

In [5]:
#Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocessing the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [6]:
# Training Model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [11]:
#Python function to chat with the chatbot
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['response'])
            return response

In [18]:
# Checking our chatbot.
user_input = "what is your age ?"
response = chatbot(user_input)
print(response)

No candles on my cake, but I'm here 24/7 to assist you


In [23]:
# Checking our chatbot.
user_input = " budget"
response = chatbot(user_input)
print(response)

Prioritize essential expenses first, then allocate funds for savings and goals.


In [24]:
# Checking our chatbot.
user_input = " how old are you "
response = chatbot(user_input)
print(response)

My knowledge is current, but I don't track time like humans do


In [27]:
# Checking our chatbot.
user_input = " bye "
response = chatbot(user_input)
print(response)

It was nice talking to you!
